# Solution to JSON exercise

By Jonas Cuadrado

May 23, 2018


## Initialization

First, we import the useful packages

In [1]:
import pandas as pd
import json
import numpy as np
import os.path

If the file is saved in /data, we can simply use the one stored. Otherwise, we will download it from the provided link

In [2]:
if os.path.exists('data/world_bank_projects.json'):
    filename = 'data/world_bank_projects.json'
    df0 = pd.read_json(filename)
    print('\nFile found, using local copy\n')
else:
    from urllib.request import urlretrieve
    from zipfile import ZipFile as ZF

    url = 'http://jsonstudio.com/wp-content/uploads/2014/02/world_bank.zip'
    file_name_zip = 'wb.zip'
    urlretrieve(url, file_name_zip)

    with ZF(file_name_zip, 'r') as zipf:
        zipf.printdir()
        zipf.extractall()
        print('\nFile downloaded\n')
    filename = 'world_bank.json'
    df0 = pd.read_json(filename, lines=True)


File found, using local copy



We are ready to explore the data

In [3]:
print(df0.columns)

Index(['_id', 'approvalfy', 'board_approval_month', 'boardapprovaldate',
       'borrower', 'closingdate', 'country_namecode', 'countrycode',
       'countryname', 'countryshortname', 'docty', 'envassesmentcategorycode',
       'grantamt', 'ibrdcommamt', 'id', 'idacommamt', 'impagency',
       'lendinginstr', 'lendinginstrtype', 'lendprojectcost',
       'majorsector_percent', 'mjsector_namecode', 'mjtheme',
       'mjtheme_namecode', 'mjthemecode', 'prodline', 'prodlinetext',
       'productlinetype', 'project_abstract', 'project_name', 'projectdocs',
       'projectfinancialtype', 'projectstatusdisplay', 'regionname', 'sector',
       'sector1', 'sector2', 'sector3', 'sector4', 'sector_namecode',
       'sectorcode', 'source', 'status', 'supplementprojectflg', 'theme1',
       'theme_namecode', 'themecode', 'totalamt', 'totalcommamt', 'url'],
      dtype='object')


We will select only two columns to perform statistics and manipulate the dataframe. Let's start by making sure there are no NaNs in country code and project id columns

In [4]:
df = df0[['countryshortname', 'mjtheme_namecode']]
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500 entries, 0 to 499
Data columns (total 2 columns):
countryshortname    500 non-null object
mjtheme_namecode    500 non-null object
dtypes: object(2)
memory usage: 11.7+ KB
None


To access the data, we need to concatenate indices... note that what we have is a list of dictionaries for the themes, some have one theme, some have many.

In [5]:
print('MJTheme')
print(df['mjtheme_namecode'].iloc[1][0])
print('NameCode')
print(df['mjtheme_namecode'].iloc[1][1])
print(type(df['mjtheme_namecode'].iloc[1][1]))

MJTheme
{'code': '1', 'name': 'Economic management'}
NameCode
{'code': '6', 'name': 'Social protection and risk management'}
<class 'dict'>


## Problem 1: Countries with most projects

Let's look at what are the lucky contries. To do so, we group the dataframe by country, aggregating using counts, and sort them. I will print 12 instead of 10, the reason will be obvious in a second.


In [6]:
cntrs = df.groupby('countryshortname').count().sort_values(by='mjtheme_namecode', ascending=False)
print(cntrs.head(12))

                    mjtheme_namecode
countryshortname                    
China                             19
Indonesia                         19
Vietnam                           17
India                             16
Yemen, Republic of                13
Nepal                             12
Bangladesh                        12
Morocco                           12
Mozambique                        11
Africa                            11
Pakistan                           9
Brazil                             9


Why is Africa there? are there projects associated to many countries?

In [13]:
africa = df0[df0['countryshortname']=='Africa']
africa.head(2)

,_id,approvalfy,board_approval_month,boardapprovaldate,borrower,closingdate,country_namecode,countrycode,countryname,countryshortname,...,sectorcode,source,status,supplementprojectflg,theme1,theme_namecode,themecode,totalamt,totalcommamt,url
45,{'$oid': '52b213b38594d8a2be17c7ad'},2014,September,2013-09-12T00:00:00Z,ECOWAS,NaN,Africa!$!3A,3A,Africa,Africa,...,JA,IBRD,Active,N,"{'Percent': 100, 'Name': 'Health system perfor...","[{'code': '67', 'name': 'Health system perform...",67,0,10000000,http://www.worldbank.org/projects/P125018/west...
46,{'$oid': '52b213b38594d8a2be17c7ae'},2014,September,2013-09-10T00:00:00Z,UGANDA-COMOROS,2018-06-30T00:00:00Z,Africa!$!3A,3A,Africa,Africa,...,"BM,CA,CT",IBRD,Active,N,"{'Percent': 20, 'Name': 'Administrative and ci...","[{'code': '25', 'name': 'Administrative and ci...","39,40,78,25",22000000,22000000,http://www.worldbank.org/projects/P118213/rcip...


The dataset doesn't seem to provide more information about which countries in Africa theose projects involve. We would need another source to know more.

## Problem 2: Major project themes

Let's use the mjcode column to count all the codes. for that, I will break the list of dictionaries into columns, break the dictionaries, melt the dataframe into a single column, and finally, group by code.

In [8]:
# this generates N columns
lst2cols = pd.DataFrame(df['mjtheme_namecode'].values.tolist())

# melt them into a single one
lst2cols = pd.melt(lst2cols)

#break the dictionaries
lst2cols2 = lst2cols['value'].apply(pd.Series)

#group, count, and rename
lst2cols2 = lst2cols2.groupby('code').count().sort_values(by='name', ascending=False)
lst2cols2.index = lst2cols2.index.astype(int)
lst2cols2 = lst2cols2.rename(columns={'name':'counts'})

# printing
print(lst2cols2)

      counts
code        
11       250
10       216
8        210
2        199
6        168
4        146
7        130
5         77
9         50
1         38
3         15


So, projects of code 3 are the least implemented, which mean that the other 10 are the most implemented. I could call head(10) to show only 10, but there are 11 in total

Now, let me create the dictionary of the names and id's, and re-print the previous information with the names associated to them.

In [9]:
# break the list into columns
listoflists = df['mjtheme_namecode'].values.tolist()

# make a single list with "sum", then create a dataframe with it
biglistdf = pd.DataFrame(sum(listoflists,[]))

# drop duplicates and nans, and empty names as well - we will replace them later
listdf = biglistdf.drop_duplicates()
listdf = listdf.dropna()
listdf = listdf[listdf['name'] != '']

# create the code - name dictionary
simpledict = dict(zip(listdf['code'].astype(int).tolist(), listdf['name'].tolist()))

# let's use ints for the code number, and set them as index
listdf['code'] = listdf['code'].astype(int)
listdf = listdf.set_index('code')

# merge the previous dataframe with the new information provided by the dictionary
sol2 = pd.concat([lst2cols2,listdf], axis=1).sort_values(by='counts', ascending=False)

# print
print(sol2.head(10))

      counts                                          name
code                                                      
11       250  Environment and natural resources management
10       216                             Rural development
8        210                             Human development
2        199                      Public sector governance
6        168         Social protection and risk management
4        146      Financial and private sector development
7        130                   Social dev/gender/inclusion
5         77                         Trade and integration
9         50                             Urban development
1         38                           Economic management


## Problem 3: Filling in missing values

The main reason to create the dictionary above is to be able to fill in the empty values on the list. There may be many ways to implement this, the least desirebale one is to loop over all rows in the dataframe. My approach here will be to define a function that replaces the empty names with names from a dictionary, for a list of dictioaries. This is applied onto de dataframe of our choice.


In [10]:
def replaceitems (listofdicts, spldict):
    # loop over elements in list
    for dct in listofdicts:
        if dct['name']=='':
            dct['name'] = spldict[int(dct['code'])]
    
    return listofdicts

df0['mjtheme_namecode'].apply(replaceitems,args=(simpledict,))
print(df0['mjtheme_namecode'].head())

0    [{'code': '8', 'name': 'Human development'}, {...
1    [{'code': '1', 'name': 'Economic management'},...
2    [{'code': '5', 'name': 'Trade and integration'...
3    [{'code': '7', 'name': 'Social dev/gender/incl...
4    [{'code': '5', 'name': 'Trade and integration'...
Name: mjtheme_namecode, dtype: object


Can we make sure this worked? Let's find all strings that contain "'name': ''" in df0. If should return an empty dataframe

In [11]:
print(df0[df0['mjtheme_namecode'].astype(str).str.contains("'name': ''")])

Empty DataFrame
Columns: [_id, approvalfy, board_approval_month, boardapprovaldate, borrower, closingdate, country_namecode, countrycode, countryname, countryshortname, docty, envassesmentcategorycode, grantamt, ibrdcommamt, id, idacommamt, impagency, lendinginstr, lendinginstrtype, lendprojectcost, majorsector_percent, mjsector_namecode, mjtheme, mjtheme_namecode, mjthemecode, prodline, prodlinetext, productlinetype, project_abstract, project_name, projectdocs, projectfinancialtype, projectstatusdisplay, regionname, sector, sector1, sector2, sector3, sector4, sector_namecode, sectorcode, source, status, supplementprojectflg, theme1, theme_namecode, themecode, totalamt, totalcommamt, url]
Index: []

[0 rows x 50 columns]


Excellent! The new dataframe is ready!

Now, feel free to save it back as json, and we call it a day.

In [12]:
df0.to_json('WorldBank_corrected.json')

Thank you so much for reading!